---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [4]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [5]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    univ_towns_df = pd.read_fwf('university_towns.txt', header=None)
    univ_towns_df_rows = univ_towns_df.shape[0]
    
    #take a new dataframe to add state and regionname columns and corresponding values
    univ_st_towns_df = pd.DataFrame(columns = ['State', 'RegionName'])
    
    # for every row check whethr edit is in the name and assign to state to if yes
    for i in range(univ_towns_df_rows):
        if 'edit' in univ_towns_df.iloc[i,0]:
            univ_st_towns_df.loc[i,'State'] = univ_towns_df.iloc[i,0]
        univ_st_towns_df.loc[i,'RegionName'] = univ_towns_df.iloc[i,0]
    
    # ffill stname so that it is copied across    
    univ_st_towns_df = univ_st_towns_df.ffill().loc[univ_st_towns_df['State'] != univ_st_towns_df['RegionName']]
             
    # For "State", removing characters from "[" to the end.
    univ_st_towns_df['State'] = univ_st_towns_df['State'].apply(lambda x: x.split('[')[0])
    univ_st_towns_df['RegionName'] = univ_st_towns_df['RegionName'].apply(lambda x: x.split('(')[0].strip() if '(' in x else x)
    
    #univ_st_towns_df['RegionName']  = univ_st_towns_df['RegionName'] .str.strip()
    
    return univ_st_towns_df

get_list_of_university_towns()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
6,Alabama,Troy
7,Alabama,Tuscaloosa
8,Alabama,Tuskegee
10,Alaska,Fairbanks
12,Arizona,Flagstaff


In [6]:
def get_recession_df():
    gdp = pd.read_excel('gdplev.xls', header=None)
    gdp = gdp.iloc[8:].reset_index(drop=True)
    
    gdp = (gdp.rename(columns={4:'Year Quarter', 5:'GDP in billions of current dollars', 6:'GDP in billions of chained 2009 dollars'})
    .drop([0,1,2,3], axis=1).set_index('Year Quarter'))


    global gdp_2000q1
    
    # only look at GDP data from the first quarter of 2000 onward
    gdp_2000q1 = gdp.loc['2000Q1':]
    
    '''
        A recession is defined as starting with two consecutive quarters of GDP decline, 
        and ending with two consecutive quarters of GDP growth. Use the chained value in 2009 dollars
    '''
    gdp_2000q1['pct_change'] = gdp_2000q1['GDP in billions of chained 2009 dollars'].pct_change()
    gdp_2000q1['Recession'] = (gdp_2000q1['pct_change'] < 0) & (gdp_2000q1['pct_change'].shift(-1) < 0)
    gdp_2000q1.reset_index(drop=True)
    
    return gdp_2000q1

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    rec_df = get_recession_df()
    return rec_df['Recession'].idxmax()
    
get_recession_start()

'2008q3'

In [7]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    '''
    A recession is defined as starting with two consecutive quarters of GDP decline, 
    and ending with two consecutive quarters of GDP growth.
    '''
    rec_df = get_recession_df()
    rec_df_true = rec_df[rec_df.Recession==True]
    rec_st = get_recession_start()
    rec_df_req = rec_df.iloc[rec_df.index.get_loc(rec_st):]
        
    for i in range(len(rec_df_req)):
        if (rec_df_req.iloc[i][1] < rec_df_req.iloc[i+1][1]) and (rec_df_req.iloc[i+1][1] < rec_df_req.iloc[i+2][1]):
            return rec_df_req.iloc[i+2].name


get_recession_end()

'2009q4'

In [8]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    rec_df = get_recession_df()
    rec_st = get_recession_start()
    rec_end = get_recession_end()
    rec_df = rec_df.loc[rec_st:rec_end]
    return rec_df['GDP in billions of chained 2009 dollars'].idxmin()

get_recession_bottom()

'2009q2'

In [13]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    '''
    A quarter is a specific three month period, Q1 is January through March, Q2 is April through June, 
    Q3 is July through September, Q4 is October through December.
    '''
#     housing_df = pd.read_csv('City_Zhvi_AllHomes.csv')
#     #housing_df.sort_values(['State', 'RegionName'], inplace=True)
#     housing_df_yrs = housing_df.loc[:,'2000-01':'2016-08']
    
#     col_list = []
#     for i in range(2000,2017):
#         col_list.append(str(i)+'q1')
#         col_list.append(str(i)+'q2')
#         col_list.append(str(i)+'q3')
#         if i != 2016:
#             col_list.append(str(i)+'q4')
    
#     cols = list(housing_df_yrs)
#     #housing_df_qts = pd.DataFrame()
#     for col in col_list:
#         i = 0
#         if col == '2016q3':
#             j = i+1
#         j = i+2
#         housing_df[col] = housing_df_yrs.iloc[:, [i,j]].mean(axis=1)
#         housing_df_yrs = housing_df_yrs.drop(housing_df_yrs.iloc[:,[i,j]], axis=1)
    
#     housing_df.drop('RegionID', axis=1, inplace=True)
#     housing_df_req = housing_df.drop(housing_df.ix[:,'Metro':'2016-08'].head(0).columns, axis=1)
#     housing_df_req = housing_df_req.replace({'State':states})
#     housing_df_req = housing_df_req.set_index(["State","RegionName"])    
#     return housing_df_req



    hdata = pd.read_csv('City_Zhvi_AllHomes.csv')
    hdata = hdata.drop(hdata.columns[[0]+list(range(3,51))],axis=1)
    hdata2 = pd.DataFrame(hdata[['State','RegionName']])
    for year in range(2000,2016):
        hdata2[str(year)+'q1'] = hdata[[str(year)+'-01',str(year)+'-02',str(year)+'-03']].mean(axis=1)
        hdata2[str(year)+'q2'] = hdata[[str(year)+'-04',str(year)+'-05',str(year)+'-06']].mean(axis=1)
        hdata2[str(year)+'q3'] = hdata[[str(year)+'-07',str(year)+'-08',str(year)+'-09']].mean(axis=1)
        hdata2[str(year)+'q4'] = hdata[[str(year)+'-10',str(year)+'-11',str(year)+'-12']].mean(axis=1)
    year = 2016    
    hdata2[str(year)+'q1'] = hdata[[str(year)+'-01',str(year)+'-02',str(year)+'-03']].mean(axis=1)
    hdata2[str(year)+'q2'] = hdata[[str(year)+'-04',str(year)+'-05',str(year)+'-06']].mean(axis=1)
    hdata2[str(year)+'q3'] = hdata[[str(year)+'-07',str(year)+'-08']].mean(axis=1)
    hdata2 = hdata2.replace({'State':states})
    hdata2 = hdata2.set_index(['State','RegionName'])
    return hdata2


#convert_housing_data_to_quarters()
convert_housing_data_to_quarters().loc["Texas"].loc["Austin"].loc["2010q3"]

# correct: 202266.66666666666

202266.66666666666

In [12]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    
#     univ_df = get_list_of_university_towns()
    
#     states_intrchngd = dict((v,k) for k,v in states.items())
#     univ_df['StateAbbr'] = univ_df['State'].map(states_intrchngd)

#     rec_st = get_recession_start()
#     rec_btm = get_recession_bottom()
    
#     housing_df_qts = convert_housing_data_to_quarters()    
#     pstart = housing_df_qts.columns[housing_df_qts.columns.get_loc(rec_st) -1]

#     housing_df_qts['chnge'] = housing_df_qts[rec_btm]-housing_df_qts[pstart]
#     #housing_df_qts['price_ratio'] = housing_df_qts[rec_btm] / housing_df_qts[rec_st]

#    # housing_df_qts['perc_chnge'] = ((housing_df_qts[rec_st]-housing_df_qts[rec_btm])/housing_df_qts[rec_st])*100

#     housing_price_rec = housing_df_qts[[rec_btm,rec_st, pstart,'chnge']].reset_index()

#     hsng_prc_univ_twns = pd.merge(housing_price_rec, univ_df, on=['State', 'RegionName'])
#     hsng_prc_univ_twns['uni'] = True
    
#     hsng_prc_non_univ_twns = housing_price_rec.merge(hsng_prc_univ_twns, how='outer', on=['State', 'RegionName', rec_btm, rec_st, 'chnge'])    
#     hsng_prc_non_univ_twns['uni'] = hsng_prc_non_univ_twns['uni'].fillna(False)
    
#     univ_twns = hsng_prc_non_univ_twns[hsng_prc_non_univ_twns['uni'] == True]
#     univ_twns['price_ratio'] = univ_twns[rec_btm] / univ_twns[rec_st]
#     non_univ_twns = hsng_prc_non_univ_twns[hsng_prc_non_univ_twns['uni'] == False]
#     non_univ_twns['price_ratio'] = non_univ_twns[rec_btm] / non_univ_twns[rec_st]

#     from scipy.stats import ttest_ind
#     t,p = ttest_ind(univ_twns['chnge'].dropna(), non_univ_twns['chnge'].dropna())
    
#     different = True if p<0.1 else False

#     better = 'university town' if univ_twns['price_ratio'].mean() < non_univ_twns['price_ratio'].mean() else 'non-university town'
        
#     return (different,p,better)

# # https://github.com/aliciasueyee/Introduction-to-Data-Science-with-Python/blob/master/Assignment%2B4.py
# run_ttest()


    unitowns = get_list_of_university_towns()
    bottom = get_recession_bottom()
    start = get_recession_start()
    hdata = convert_housing_data_to_quarters()
    bstart = hdata.columns[hdata.columns.get_loc(start) -1]
    
    hdata['ratio'] = hdata[bottom] - hdata[bstart]
    hdata = hdata[[bottom,bstart,'ratio']]
    hdata = hdata.reset_index()
    unitowns_hdata = pd.merge(hdata,unitowns,how='inner',on=['State','RegionName'])
    unitowns_hdata['uni'] = True
    hdata2 = pd.merge(hdata,unitowns_hdata,how='outer',on=['State','RegionName',bottom,bstart,'ratio'])
    hdata2['uni'] = hdata2['uni'].fillna(False)

    ut = hdata2[hdata2['uni'] == True]
    nut = hdata2[hdata2['uni'] == False]

    t,p = ttest_ind(ut['ratio'].dropna(),nut['ratio'].dropna())
    
    different = True if p < 0.01 else False

    better = "non-university town" if ut['ratio'].mean() < nut['ratio'].mean() else "university town"

    return different, p, better

run_ttest()

(True, 0.002099659657952052, 'university town')